# SPARK ML
MLlib es una biblioteca básica de Spark que proporciona numerosas utilidades para las tareas de aprendizaje automático, por ejemplo: clasificación. Regresión. Agrupación en clústeres.

In [0]:
# definir sesión de spark
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark import SparkFiles
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sesion007").getOrCreate()

## REGRESIÓN LINEA SIMPLE
La regresión lineal es un método estadístico fundamental utilizado para modelar la relación entre una variable dependiente (también llamada variable respuesta o variable objetivo) y una o más variables independientes (también llamadas predictores o variables explicativas). En términos simples, busca establecer una relación lineal que explique cómo una variable dependiente cambia en función de una o más variables independientes.

documentación de referencia: https://spark.apache.org/docs/latest/ml-guide.html

### LIBRERÍAS REQUERIDAS

### DATASET

Estos datos representan la edad (en años) y el salario (en miles de dólares) correspondiente. Cada fila en la tabla representa una observación diferente, donde cada persona tiene una edad específica y un salario asociado.

| age | salary |
|-----|--------|
| 25  | 50     |
| 30  | 60     |
| 35  | 70     |
| 40  | 80     |
| 45  | 90     |
| 50  | 100    |
| 55  | 110    |
| 60  | 120    |
| 65  | 130    |
| 70  | 140    |

Los datos buscan responder cómo varía el salario en función de la edad. En este caso específico:

1. Relación entre edad y salario: La regresión lineal nos permitiría modelar esta relación para entender si existe una tendencia lineal en cómo el salario cambia a medida que la edad aumenta.

3. Predicciones de Salario: Una vez ajustado el modelo, podríamos hacer predicciones sobre el salario esperado para edades no observadas en nuestros datos originales. Esto sería útil para entender cómo el salario podría cambiar en función de la edad en el futuro.

3. Impacto de la Edad en el Salario: Los coeficientes obtenidos del modelo de regresión lineal nos darían una idea del impacto promedio de cada año adicional de edad en el salario. Esto nos ayuda a cuantificar y entender la relación entre estas variables.

In [0]:
url: str = "https://gitlab.com/luisvasv/public/-/raw/master/datasets/spark/salaries.txt"
spark.sparkContext.addFile(url)

spark_uri: str = "file://"+SparkFiles.get("salaries.txt")
print(spark_uri)

### IMPLEMENTACION

#### LECTURA DE DATOS

In [0]:
df = spark.read.option("header", True).option("inferSchema", True).csv(spark_uri)
df.printSchema()
df.show()

#### VECTOR ASSEMBLER

### ¿Qué es el VectorAssembler?

- **Propósito**: Combina múltiples columnas de un DataFrame en un solo vector.
- **Importancia**: Necesario para modelos de machine learning en PySpark que requieren una única entrada de características.

### ¿Por qué se Usa?

- **Requisito de Modelos ML**: Algunos algoritmos (como regresión lineal, regresión logística) necesitan que las características estén en un vector.
- **Simplificación de Datos**: Prepara los datos para modelos, mejorando la eficiencia y compatibilidad con pipelines de ML.


In [0]:
assembler = VectorAssembler(inputCols=['age'], outputCol='features')
df = assembler.transform(df)
df.show()

####  DEFINIR SETS

In [0]:
# seleccionamos los datos
data = df.select('features', 'salary')

# se dividen los datos: 80% para entrenamiento, 20% para prueba (valores ajustables)
train_data, test_data = data.randomSplit([0.8, 0.2])

#### DEFINIR Y ENTRENAR MODELO

In [0]:
# definición del modelo
lr = LinearRegression(featuresCol='features', labelCol='salary')

# entrenamioento del modelo
lr_model = lr.fit(train_data)

#### RESULTADOS

###### COEFICIENTES

El coeficiente (o coeficientes, si hay más de una característica) representa la magnitud del cambio esperado en la variable dependiente (en este caso, el salario) por cada unidad de cambio en la variable independiente (edad).

**INTERPRETACIÓN**: el coeficiente es `[2.0]`. Esto significa que, según el modelo ajustado, se espera que el salario aumente en 2 unidades de la moneda (por ejemplo, miles de dólares) por cada unidad adicional de edad.

**SIGNIFICADO**: Un coeficiente positivo indica una relación directa entre la variable independiente y la variable dependiente. Es decir, a medida que aumenta la edad, el modelo predice que el salario también aumentará.

In [0]:
print(f"coeficiente(s) ----> : {lr_model.coefficients}")

###### INTERCEPTO

El intercepto es el valor esperado de la variable dependiente cuando todas las variables independientes son cero. En el contexto de regresión lineal, representa el valor inicial de la variable dependiente antes de considerar cualquier efecto de las variables independientes.

**INTERPRETACIÓN**: Cuando la edad de un empleado es 0 años, el modelo predice que su salario sería de 0 unidades de la moneda (por ejemplo, 0 dólares).

In [0]:
print(f"intercepto.    ----> : {lr_model.intercept}")

###### PREDICCIONES

In [0]:
# creamos unos datos aleatorios para predecir el salario a partir de la edad
new_ages = [(20.0,), (55.0,), (58.0,), (63.0,)]
df_ages = spark.createDataFrame(new_ages, ["age"])
df_ages = assembler.transform(df_ages)

predictions = lr_model.transform(df_ages)
predictions.select("features", "prediction").show()

